# Mise à disposition du jeu de données stroke_dataset via une API REST

## Description du dataset

Le jeu de données utilisé provient de Kaggle : [Stroke Prediction Dataset](https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset).  
Il contient des données de patients avec différentes caractéristiques médicales et sociales, ainsi que l'information si le patient a subi un accident vasculaire cérébral (AVC) ou non.

Télécharger les données et ajouter les dans un dossier data/.

Les colonnes des données sont :  
- `id` : Identifiant unique du patient  
- `gender` : Sexe  
- `age` : Âge  
- `hypertension` : Présence d'hypertension (0 ou 1)  
- `heart_disease` : Présence de maladie cardiaque (0 ou 1)  
- `ever_married` : Statut marital  
- `work_type` : Type d'emploi  
- `Residence_type` : Urbaine ou rurale  
- `avg_glucose_level` : Moyenne du taux de glucose  
- `bmi` : Indice de masse corporelle  
- `smoking_status` : Statut tabagique  
- `stroke` : Présence d'AVC (0 ou 1)

## Projet

Vous devez exposer les données patients du jeu de données via une API REST afin que les données soit utilisables par d'autres équipes (médecins, data science, étude, etc.).

Cette API REST sera développée avec FastAPI et les spécifications sont les suivantes :
| Méthode | Endpoint                                      | Fonctionnalité                                                                                                    |
| ------- | --------------------------------------------- | ----------------------------------------------------------------------------------------------------------------- |
| `GET`   | `/patients/{id}`                              | Récupère les détails d’un patient donné (via son identifiant unique)                                              |
| `GET`   | `/patients?stroke=1&gender=Female&max_age=60` | Renvoie les patients filtrés selon plusieurs critères : AVC (oui/non), genre, âge maximal                         |
| `GET`   | `/stats/`                                     | Fournit des statistiques agrégées sur les patients (ex. : nb total de patients, âge moyen, taux d’AVC, répartition hommes/femmes, etc.) |



---

## Quelques définitions


1. Qu’est-ce qu’une API REST ?

- API signifie Application Programming Interface (Interface de Programmation d’Application). C’est un ensemble de règles et de protocoles qui permettent à des logiciels de communiquer entre eux.
- REST signifie Representational State Transfer. C’est un style architectural pour concevoir des services web.
Il en existe d'autres mais REST est celui que vous rencontrerez le plus souvent.
- Vous avez utilisé une API REST via l'API Google Books.

- A quoi sert une API REST ?

    - Permet à différentes applications de communiquer facilement, même si elles sont écrites dans des langages différents.
    - Permet d’accéder à des services distants (ex : bases de données, services web) de manière standardisée.
    - Facilite la création d’applications modulaires et évolutives (front-end, back-end, mobile, etc.)

2. Principes clés d’une API REST

- a. Utilisation du protocole HTTP
Les échanges entre client et serveur utilisent des méthodes HTTP standard comme :

    - GET : pour récupérer des données
    - POST : pour envoyer ou créer des données
    - PUT : pour mettre à jour des données
    - DELETE : pour supprimer des données

- b. Accès aux ressources via des URLs

Chaque ressource (par exemple un livre, un utilisateur) est accessible via une URL unique.

Exemple fictif:
    https://api.example.com/books/123 pour accéder au livre d’identifiant 123.

- c. Stateless (sans état)

Le serveur ne conserve aucune information sur le client entre deux requêtes. Chaque requête doit contenir toutes les informations nécessaires.

- d. Représentations des données

Les données sont envoyées et reçues généralement en format JSON ou XML, qui sont faciles à lire et à manipuler.

- e. Utilisation de codes status HTTP

Chaque réponse du serveur est accompagnée d’un code HTTP indiquant le résultat de la requête. (cf [liste des codes](https://developer.mozilla.org/en-US/docs/Web/HTTP/Reference/Status))

## Outils utilisés
1. FastAPI

FastAPI est un framework Python moderne, rapide et très utilisé dans le milieu professionnel pour construire des API REST. 

Il permet :
- une définition simple des routes et des paramètres  
- La génération automatique de documentation interactive (Swagger UI)  
- FastAPI lit les requêtes entrantes, les traite avec ton code Python, et retourne une réponse HTTP (en JSON).

2. Uvicorn : exécute l'application FastAPI

- Uvicorn est un serveur ASGI (Asynchronous Server Gateway Interface) : c'est une interface standard pour gérer les requêtes de manière asynchrone et performante, notamment utile pour les applications modernes.
- Il attend les requêtes HTTP (par exemple depuis un navigateur), les transmet à FastAPI, et renvoie la réponse.
- Uvicorn permet à l'API de fonctionner : sans Uvicorn ou un autre serveur, FastAPI ne peut pas fonctionner.


3. Swagger UI : l’interface de doc et test interactive

- Swagger UI est généré automatiquement par FastAPI.
- C’est une interface web qui permet de :
    - Voir toutes les routes disponibles dans l'API
    - Tester les routes en envoyant des requêtes sans écrire de code (bouton try it out)
    - Voir les paramètres attendus et les formats de réponse
    
4. Résumé des interactions

- Tester la route de base de l'API grâce à la commande :
```bash
    poetry run fastapi dev stroke_api/main.py
```

--> Qu'est-ce qu'il se passe derrière cette commande ?

- Uvicorn démarre un serveur local
- FastAPI génère automatiquement une interface : Swagger UI, accessible sur http://127.0.0.1:8000/docs qui affiche toutes les routes définies dans le code python FastAPI
- Quand on clique sur "Try it out" dans Swagger UI, Swagger envoie une requête HTTP au serveur (ici Uvicorn)
- Le serveur (Uvicorn) la reçoit, l’envoie à FastAPI, qui traite et renvoie une réponse
- Swagger UI affiche la réponse de l’API (par ex : liste de patients)

---

Import des bibliothèques utiles au projet

In [ ]:
# import pandas library to start working on our dataset 
import pandas as pd

## 1. Prétraitement des données / Data preprocessing

Les données réelles sont rarement prêtes à être utilisées directement. Elles peuvent contenir des erreurs, des valeurs manquantes, des doublons, des formats incohérents, ou ne pas être adaptées au modèle ou au système cible.

Le prétraitement consiste à nettoyer, structurer et transformer les données brutes avant de les exploiter dans un projet (modèle IA, API, visualisation, etc.).

Vous avez déjà prétraité des données, petit rappel des éléments sur lesquels travailler dans un prétraitment classique et les méthodes pandas qu'il est possible d'utiliser pour les différentes étapes (des exeples d'utilisation des méthodes pandas sont disponibles dans la doc) : 
- explorer les données pour identifier les types de données, valeurs manquantes, incohérence ([info](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.info.html), [dtypes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dtypes.html))
- adapter les types si nécessaire ([astypes](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html))
- identifier les doublons et les supprimer s'il y en a ([duplicated](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html), [drop_duplicates](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html))
- traiter les valeurs manquantes s'il y en a ([fillna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html), [dropna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html), [replace](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html))
- identifier les incohérences éventuelles (valeurs aberrantes/outliers) en vérifiant si les valeurs min, max, moyennes sont raisonnables (recherche internet si nécessaire) ([describe](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html)), et les traiter.
- Traiter les valeurs aberrantes si vous en détectez ([loc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html) pour récupérer les lignes qui répondent à une certaine condition, cf exemple ci-dessous)


**Exemple df.loc :**

Récupérer toutes les lignes de df telles que la valeur de "nom de colonne" >= 0

```df_subset = df.loc[stroke_data_df['nom de colonne'] >= 0]```



---
### **TODO**
1.a. Prétraiter les données du dataset.


1.b. Documenter dans le README.md :
- Les étapes de prétraitement,
- Justification des choix concernant le traitement des valeurs manquantes (si besoin),
- Liste des valeurs raisonnables utilisées pour détecter les valeurs aberrantes, 
- Justification des choix pour traiter les valeurs aberrantes (si besoin).

2.a. Chercher des infos sur le format de fichier parquet et indiquer les sources consultées : 
- Différence principale avec le format csv ?
- Dans quels cas l'utiliser ? 
- Pourquoi c'est un format adapté aux gros volumes de données ?



2.b. Sauvegarder les données prétraiteées dans un fichier parquet ([to_parquet](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_parquet.html)).


                            #################################################################################################################
                            ############################################ Initial EDA ##########################################################
                            #################################################################################################################


In [5]:
# Prétraitement de données
# load and disply the health data before start the pretreatement 
df_health = pd.read_csv("data/healthcare-dataset-stroke-data.csv")
df_health.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [6]:
# display basic ifno about the dataset
df_health.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


In [10]:
# display descriptive statistics of the dataset 
df_health.describe()

,id,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke
count,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000,4909.000000,5110.000000
mean,36517.829354,43.226614,0.097456,0.054012,106.147677,28.893237,0.048728
std,21161.721625,22.612647,0.296607,0.226063,45.283560,7.854067,0.215320
min,67.000000,0.080000,0.000000,0.000000,55.120000,10.300000,0.000000
25%,17741.250000,25.000000,0.000000,0.000000,77.245000,23.500000,0.000000
50%,36932.000000,45.000000,0.000000,0.000000,91.885000,28.100000,0.000000
75%,54682.000000,61.000000,0.000000,0.000000,114.090000,33.100000,0.000000
max,72940.000000,82.000000,1.000000,1.000000,271.740000,97.600000,1.000000


In [11]:
# identify the Max age value in the dataset
df_health['age'].max()

np.float64(82.0)

In [12]:
# Count and filter missing data
def count_filter(df):
    print('total missing data by their columns:')
    #count missing values per column 
    missing_values = df.isna().sum()
    #filter only columns with at leats one missing values 
    missing_count = missing_values[missing_values > 0]
    
    return missing_count
count_filter(df_health)

total missing data by their columns:


bmi    201
dtype: int64

In [13]:
# check for non-logical vales in ID column 
# assuming id can't be 0, NaN, negative value, float or string
def check_id(df):
    # Check for NaN values or IDs less than or equal to 0
    invalid_num = df['id'].isna() | (df['id'] <= 0)
    
    # Check for IDs that are not integers
    # Create a boolean mask identifying values that are NOT integers in the 'id' column
    invalid_type = ~df['id'].apply(lambda x: isinstance(x, int))
    
    # Combine both conditions to find invalid IDs
    wrong_id = invalid_num | invalid_type
    
    # Count how many invalid IDs there are
    num_r_id = wrong_id.sum()
    
    print(f"There are: {num_r_id} rows with non-logical id values")
    return num_r_id

check_id(df_health)

There are: 0 rows with non-logical id values


np.int64(0)

In [14]:
# check for gender column 
# assuming vales should be only male, female or 'other' For people who don't want to disclose their gender 
def check_gender(df):
    valid_gender = ['Male', 'Female', 'Other'],
    wrong_gender = df['gender'].isin(valid_gender)
    n_r_gender = wrong_gender.sum()
    print(f"There are: {n_r_gender} with non-logical gender values")
    
    return n_r_gender
check_gender(df_health)

There are: 0 with non-logical gender values


np.int64(0)

In [15]:
# check for the non-logical data in age column 
# assuming age should not be nigative 
def check_age(df) :
    wrong_age = ((df['age'] < 0) | (df['age'] > 100))
    # How many ?
    num_r_age = wrong_age.sum()
    print(f"There are: {num_r_age} rows with non-logical age values")
    return num_r_age
check_age(df_health)

There are: 0 rows with non-logical age values


np.int64(0)

In [16]:
# check for the non-logical data in hypertension column 
# assuming hypertension values should be only 1 or 0
def check_hypertension(df):
    wrong_hypertension = ((df['hypertension'] != 1) & (df['hypertension'] != 0))
    #how many ?
    num_r_hypertension = wrong_hypertension.sum()
    print(f"There are: {num_r_hypertension} rows with non-logical hypertension values ")
    return num_r_hypertension
check_hypertension(df_health)

There are: 0 rows with non-logical hypertension values 


np.int64(0)

In [17]:
# check for non-logical vales in heart_disease column 
# assuming this column's values should be only 1 or 0 
def check_heart_disease(df):
    # Check for values in 'heart_disease' column that are not 0 or 1
    wrong_heart_disease = (df['heart_disease'] != 0) & (df['heart_disease'] != 1)
    
    # Count how many invalid heart_disease values
    num_r_heart_disease = wrong_heart_disease.sum()
    
    print(f"There are: {num_r_heart_disease} rows with non-logical heart_disease values")
    return num_r_heart_disease

check_heart_disease(df_health)

There are: 0 rows with non-logical heart_disease values


np.int64(0)

In [18]:
# check for the non-logical data in ever_married column 
# assuming ever_married column should not be yes or no 
def check_marriage(df):
    # Check for values in 'ever_married' column that are NOT 'Yes' or 'No'
    wrong_marriage = (df['ever_married'] != 'Yes') & (df['ever_married'] != 'No')
    
    # Count how many invalid marriage status values
    num_r_marriage = wrong_marriage.sum()
    
    print(f"There are: {num_r_marriage} rows with non-logical marriage values")
    return num_r_marriage

check_marriage(df_health)

There are: 0 rows with non-logical marriage values


np.int64(0)

In [19]:
# check for non-logical values in work-type column 
# 1- check unique vlaues to use them as valid types 
# 2- check for values are not in the list of valid types
# output should be false 
def check_work_type(df):
    valid_work_types = df['work_type'].unique()
    wrong_work_type = ~df['work_type'].isin(valid_work_types)
    return wrong_work_type
check_work_type(df_health)

0       False
1       False
2       False
3       False
4       False
        ...  
5105    False
5106    False
5107    False
5108    False
5109    False
Name: work_type, Length: 5110, dtype: bool

In [21]:
# check for the non-logical data in Residence_type column 
# assuming Residence_type column should be Ubran  or Rural 
def check_Residence_type(df):
    valid_Residence_type = df['Residence_type'].unique()
    wrong_Residence_type = ~df['work_type'].isin(valid_Residence_type)
    return wrong_Residence_type
check_work_type(df_health)

0       False
1       False
2       False
3       False
4       False
        ...  
5105    False
5106    False
5107    False
5108    False
5109    False
Name: work_type, Length: 5110, dtype: bool

In [22]:
# Assuming avg_glucose_level should be a positive number and reasonable max (e.g., less than 500)
def check_avg_glucose(df):
    wrong_glucose = ((df['avg_glucose_level'] <= 0) | (df['avg_glucose_level'] > 300))
    print(f"There are {wrong_glucose.sum()} rows with non-logical avg_glucose_level values")
    return wrong_glucose
check_avg_glucose(df_health)

There are 0 rows with non-logical avg_glucose_level values


0       False
1       False
2       False
3       False
4       False
        ...  
5105    False
5106    False
5107    False
5108    False
5109    False
Name: avg_glucose_level, Length: 5110, dtype: bool

In [23]:
# checking non-logical values in bmi column 
# assuming that bmi can't be 0 or negative value and reasonalbe max less than 60 
def check_bmi(df):
    wrong_bmi = (df['bmi'] <= 0) | (df['bmi'] > 60)
    print(f"There are {wrong_bmi.sum()} rows with non-logical bmi values")
    # Filter and print the non-logical bmi values
    #print(df.loc[(df['bmi'] <= 0) | (df['bmi'] > 60), 'bmi'])
    return wrong_bmi
check_bmi(df_health)
# for now i let my data like this because i have no clu about bmi lvls

There are 13 rows with non-logical bmi values


0       False
1       False
2       False
3       False
4       False
        ...  
5105    False
5106    False
5107    False
5108    False
5109    False
Name: bmi, Length: 5110, dtype: bool

In [ ]:
# display unique values of smoke status column to use it as valid values 
df_health['smoking_status'].unique()

array(['formerly smoked', 'never smoked', 'smokes', 'Unknown'],
      dtype=object)

In [27]:
# Checking for non-logical values in smoking_status column
# Assuming that the valid values are: 'formerly smoked', 'never smoked', 'smokes'

def check_smoking_status(df):
    # Define valid smoking status values
    valid_values = ['formerly smoked', 'never smoked', 'smokes']
    
    # Find rows where smoking_status is NOT in the valid values
    wrong_smoking = ~df['smoking_status'].isin(valid_values)
    
    # Count how many invalid smoking_status values
    num_wrong = wrong_smoking.sum()
    
    # Print results
    print(f"There are: {num_wrong} rows with non-logical smoking_status values")
    
    # Show unique invalid values (optional for debugging)
    if num_wrong > 0:
        invalid_values = df.loc[wrong_smoking, 'smoking_status'].unique()
        print("Invalid values found:", invalid_values)
    
    return num_wrong

# Call the function
check_smoking_status(df_health)


There are: 1544 rows with non-logical smoking_status values
Invalid values found: ['Unknown']


np.int64(1544)

In [28]:
# check for the non-logical data in stroke column 
# assuming stroke values should be only 1 or 0
def check_stroke(df):
    wrong_stroke = ((df['stroke'] != 1) & (df['stroke'] != 0))
    #how many ?
    num_r_stroke = wrong_stroke.sum()
    print(f"There are: {num_r_stroke} rows with non-logical hypertension values ")
    return num_r_stroke
check_stroke(df_health)

There are: 0 rows with non-logical hypertension values 


np.int64(0)

In [29]:
# check for duplicated rows in dataset
def duplicated(df):
    dup = df.duplicated().sum()
    print(f"There are: {dup} duplicated rows")
    return dup
duplicated(df_health)

There are: 0 duplicated rows


np.int64(0)

                                #################################################################################################################
                                ############################################ Data Cleaning ##########################################################
                                #################################################################################################################


In [30]:
# create a copy of the dataset to clean it without modifing the original
df_imputed = df_health.copy()

In [23]:
# # impute the misssing values in bmi columns by the mean bmi filterd by gender and age 
# def impute_bmi(df):
#     """ return a copy of df where missing bmi values have been 
#     replaced with the mean bmi filtered by gender and age """
#     group_means = df.groupby(['age', 'gender'])['bmi'].transform('mean')
#     df['bmi'] = df['bmi'].fillna(group_means)
#     return df


# df_imputed = impute_bmi(df_imputed)

In [24]:
# # apply a tset to check if all values have been imputed successfully 
# nan_rows = df_imputed[df_imputed['bmi'].isna()]
# print(nan_rows)
# print("#" * 70)
# row_index = nan_rows.index[0]
# print(row_index)

In [25]:
# # after i found we still have one row with NaN value in the bmi column 
# # i decided to treat it individually 

# def impute_individual(df, gender, age):
#     """ return a copy of df where missing bmi values have been 
#     replaced with the mean bmi filtered by ( where gender is male and age is less than 1 year)"""
#     # mean bmi for male under 1 years old 
#     #mean_individual = df[(df['gender'] == 'Male') & (df['age'] < 1)]['bmi'].mean()
#     mean_individual = df[(df['gender'] == gender) & (df['age'] < age + 1 ) & (df['age'] >= age - 1)]['bmi'].mean()
#     # we detect the row with NaN value 
#     nan_rows = df_imputed[df_imputed['bmi'].isna()]
#     row_index = nan_rows.index[0]
#     # we impute that value with the mean individual value we calculated
#     df.at[row_index, 'bmi'] = mean_individual
#     return df
# df_imputed = impute_individual(df_imputed, 'male', 0.48)

In [31]:
# This function imputes missing BMI values by calculating the mean BMI 
# of individuals with the same gender and similar age (± window).
# If BMI is not missing, it returns the original value.
# The function is applied row-wise to the dataframe.
def compute_bmi_mean(row, df, window=1):
    if pd.isna(row['bmi']):
        gender = row['gender']
        age = row['age']
        # filter group with similar gender and close age
        mask = (df['gender'] == gender) & (df['age'] >= age - window) & (df['age'] <= age + window) & (df['bmi'].notna())
        mean_bmi = df.loc[mask, 'bmi'].mean()
        return mean_bmi
    else:
        return row['bmi']

# apply to whole dataframe
df_imputed['bmi'] = df_imputed.apply(lambda row: compute_bmi_mean(row, df_imputed), axis=1)


In [33]:
# check if we imputed the missing values  successfully 
df_imputed.isna().sum()

id                   0
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

In [ ]:
# save the cleaned data to parquet
#df_imputed.to_parquet("data/clean_health.parquet")

-----
## Développement de l'API

A présent que les données sont propres, on peut débuter la création de l'API.

Pour cela, vous allez avoir besoin de quelques fonctions permettant de filtrer les données.

Vous allez les définir ci-dessous, ce qui vous permettra de les tester puis les fonctions seront reportées dans le fichier filters.py.

## Route `/patients/`
- Cette route retourne une liste filtrée de patients
- On souhaite pouvoir filtrer par `gender`, `stroke` ou `max_age`

L'objectif est ici de définir une fonction python qui prend en entrée les paramètres optionnels : _gender_, *stroke*, *max_age* et qui renvoie un dictionnaire filtré des données.

On décompose la rédaction de cette fonction en plusieurs étapes. 

Dans un premier temps, écrire et tester les filtres que l'on souhaite appliquer sur les données (utiliser [loc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html))

In [35]:
# Filtrer le dataframe pour ne garder que les patients pour lesquels "stroke=1"
def patients_with_stoke(df):
    df = df.loc[df['stroke'] == 1]
    return df
filtered_data = patients_with_stoke(df_imputed)
filtered_data

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.600000,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,30.970085,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.500000,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.400000,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.000000,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
244,17739,Male,57.0,0,0,Yes,Private,Rural,84.96,36.700000,Unknown,1
245,49669,Female,14.0,0,0,No,children,Rural,57.93,30.900000,Unknown,1
246,27153,Female,75.0,0,0,Yes,Self-employed,Rural,78.80,29.300000,formerly smoked,1
247,34060,Male,71.0,1,0,Yes,Self-employed,Rural,87.80,30.381967,Unknown,1


In [36]:
# Filtrer les données pour ne garder que les patients pour lesquels "gender="male"
def patients_males(df):
    df = df.loc[df['gender'] == 'Male']
    return df
males_data = patients_males(df_imputed)
males_data

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.600000,formerly smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.500000,never smoked,1
5,56669,Male,81.0,0,0,Yes,Private,Urban,186.21,29.000000,formerly smoked,1
6,53882,Male,74.0,1,1,Yes,Private,Rural,70.09,27.400000,never smoked,1
13,8213,Male,78.0,0,1,Yes,Private,Urban,219.84,27.440845,Unknown,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5097,64520,Male,68.0,0,0,Yes,Self-employed,Urban,91.68,40.800000,Unknown,0
5098,579,Male,9.0,0,0,No,children,Urban,71.88,17.500000,Unknown,0
5099,7293,Male,40.0,0,0,Yes,Private,Rural,83.94,31.411688,smokes,0
5100,68398,Male,82.0,1,0,Yes,Self-employed,Rural,71.97,28.300000,never smoked,0


In [37]:
# Filtrer les données pour ne garder que les patients tels que "age <= max_age"
def filter_max_age(df):
    df = df.loc[df['age'] <= df['age'].max()]
    return df
max_age_data = filter_max_age(df_imputed)
max_age_data

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.600000,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,30.970085,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.500000,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.400000,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.000000,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,28.145455,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.000000,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.600000,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.600000,formerly smoked,0


Appliquer successivement les 3 filtres au sein d'une fonction qui prend en entrée le dataframe, _stroke_, _gender_, _max_age_ et qui renvoie une liste de dictionnaire de patients (utiliser la méthode pandas [to_dict](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_dict.html)).

Exemple
```
[{'id': 9046,
  'gender': 'Male',
  'age': 67.0,
  ...
  'smoking_status': 'formerly smoked',
  'stroke': 1},
 {'id': 31112,
  'gender': 'Male',
  'age': 80.0,
  ...
  'smoking_status': 'formerly smoked',
  'stroke': 1}]
  ```

In [38]:
def filter_patient(df, max_age, gender, stroke)->dict:
    df = df.loc[(df['stroke'] == stroke )& (df['gender'] == gender) & (df['age'] <= max_age)]
    return df.to_dict()
filter_patient(df_imputed, 56, 'Female', 1)

{'id': {3: 60182,
  12: 12175,
  15: 58202,
  21: 13861,
  39: 62602,
  49: 36338,
  59: 5111,
  60: 10710,
  63: 19557,
  74: 19773,
  78: 45805,
  106: 42072,
  107: 12062,
  109: 59125,
  113: 41069,
  118: 31720,
  121: 72918,
  133: 31563,
  154: 56939,
  156: 43054,
  162: 69768,
  166: 63453,
  180: 54567,
  182: 39912,
  210: 33943,
  211: 62439,
  213: 66866,
  217: 29552,
  228: 31154,
  238: 1836,
  245: 49669},
 'gender': {3: 'Female',
  12: 'Female',
  15: 'Female',
  21: 'Female',
  39: 'Female',
  49: 'Female',
  59: 'Female',
  60: 'Female',
  63: 'Female',
  74: 'Female',
  78: 'Female',
  106: 'Female',
  107: 'Female',
  109: 'Female',
  113: 'Female',
  118: 'Female',
  121: 'Female',
  133: 'Female',
  154: 'Female',
  156: 'Female',
  162: 'Female',
  166: 'Female',
  180: 'Female',
  182: 'Female',
  210: 'Female',
  211: 'Female',
  213: 'Female',
  217: 'Female',
  228: 'Female',
  238: 'Female',
  245: 'Female'},
 'age': {3: 49.0,
  12: 54.0,
  15: 50.0,
  21:

A présent on souhaite ajouter des informations sur les types des paramètres et valeurs de retour de la fonction pour faciliter sa compréhension et son utilisation, ce qu’on appelle l’annotation de type (type hinting).

Cette pratique facilite la lecture et la maintenance du code.

Quels changements pour la fonction ?

A la suite de chaque paramètre, on ajoute le type attendu pour le paramètre. À la suite des paramètres on ajoute le type de ce que qui est retourné par la fonction, dans l'exemple ici : 

```def filter_patient(stroke_data_df: pd.DataFrame, gender: str, etc) -> list[dict]```

Ajouter les types dans la définition de la fonction.

Tester la fonction en ne mettant pas de valeur pour *max_age*.

Que se passe-t-il ?

In [39]:
def filter_patient(df: pd.DataFrame, max_age: int, gender: str, stroke: int)->list[dict]:
    df = df.loc[(df['stroke'] == stroke )& (df['gender'] == gender) & (df['age'] <= max_age)]
    return df.to_dict(orient="records")
filter_patient(df_imputed, 56, 'Female', 1)

[{'id': 60182,
  'gender': 'Female',
  'age': 49.0,
  'hypertension': 0,
  'heart_disease': 0,
  'ever_married': 'Yes',
  'work_type': 'Private',
  'Residence_type': 'Urban',
  'avg_glucose_level': 171.23,
  'bmi': 34.4,
  'smoking_status': 'smokes',
  'stroke': 1},
 {'id': 12175,
  'gender': 'Female',
  'age': 54.0,
  'hypertension': 0,
  'heart_disease': 0,
  'ever_married': 'Yes',
  'work_type': 'Private',
  'Residence_type': 'Urban',
  'avg_glucose_level': 104.51,
  'bmi': 27.3,
  'smoking_status': 'smokes',
  'stroke': 1},
 {'id': 58202,
  'gender': 'Female',
  'age': 50.0,
  'hypertension': 1,
  'heart_disease': 0,
  'ever_married': 'Yes',
  'work_type': 'Self-employed',
  'Residence_type': 'Rural',
  'avg_glucose_level': 167.41,
  'bmi': 30.9,
  'smoking_status': 'never smoked',
  'stroke': 1},
 {'id': 13861,
  'gender': 'Female',
  'age': 52.0,
  'hypertension': 1,
  'heart_disease': 0,
  'ever_married': 'Yes',
  'work_type': 'Self-employed',
  'Residence_type': 'Urban',
  'avg

Dans la fonction écrite ci-dessus, chaque paramètre est obligatoire. 

On souhaite pouvoir filtrer les patients sur 0, 1 ou 2 des paramètres de la fonction (filtrer seulement sur *max_age*  mais ne pas appliquer de filtres sur _gender_ et _stroke_ par exemple).

On peut rendre optionnel les paramètres d'un fonction en choisissant une valeur par défault. Si on utilise la fonction en n'utilisant pas ces paramètres alors la valeur par défault est utilisé.

Copier coller votre fonction ci-dessous et ajouter en paramètre : `max_age=None`

et ajouter la condition suivante **avant le filtre** sur `max_age` : 

```if max_age is not None : ``` 

Si la fonction _filter_patient_ est appelée sans argument *max_age*, alors le filtre sur *max_age* n'est pas appliqué. 

Il est tout à fait possible de définir une valeur par défault par exemple 30 ans : dans ce cas si la fonction est appelée sans argument *max_age*, alors par défault on filtre les patients ayant moins de 30 ans.

**ATTENTION :** Les paramètres optionnels doivent toujours être à la fin de la liste de paramètres.

In [40]:
# fonction filter_patient paramètre max_age optionnel
def filter_patient(df: pd.DataFrame, gender: str, stroke: int, max_age=None)->list[dict]:
    df = df.loc[(df['stroke'] == stroke ) & (df['gender'] == gender)]
    if max_age is not None : 
        df = df.loc[df['age'] <= max_age]
    return df.to_dict(orient="records")

filter_patient(df_imputed, 'Female', 1, 30)

[{'id': 69768,
  'gender': 'Female',
  'age': 1.32,
  'hypertension': 0,
  'heart_disease': 0,
  'ever_married': 'No',
  'work_type': 'children',
  'Residence_type': 'Urban',
  'avg_glucose_level': 70.37,
  'bmi': 18.44230769230769,
  'smoking_status': 'Unknown',
  'stroke': 1},
 {'id': 49669,
  'gender': 'Female',
  'age': 14.0,
  'hypertension': 0,
  'heart_disease': 0,
  'ever_married': 'No',
  'work_type': 'children',
  'Residence_type': 'Rural',
  'avg_glucose_level': 57.93,
  'bmi': 30.9,
  'smoking_status': 'Unknown',
  'stroke': 1}]

In [41]:
# test fonction sans argument max_age
def filter_patient(df: pd.DataFrame, gender: str, stroke: int, max_age=None)->list[dict]:
    df = df.loc[(df['stroke'] == stroke ) & (df['gender'] == gender)]
    if max_age is not None : 
        df = df.loc[df['age'] <= max_age]
    return df.to_dict(orient="records")

filter_patient(df_imputed, 'Female', 1)

[{'id': 51676,
  'gender': 'Female',
  'age': 61.0,
  'hypertension': 0,
  'heart_disease': 0,
  'ever_married': 'Yes',
  'work_type': 'Self-employed',
  'Residence_type': 'Rural',
  'avg_glucose_level': 202.21,
  'bmi': 30.97008547008547,
  'smoking_status': 'never smoked',
  'stroke': 1},
 {'id': 60182,
  'gender': 'Female',
  'age': 49.0,
  'hypertension': 0,
  'heart_disease': 0,
  'ever_married': 'Yes',
  'work_type': 'Private',
  'Residence_type': 'Urban',
  'avg_glucose_level': 171.23,
  'bmi': 34.4,
  'smoking_status': 'smokes',
  'stroke': 1},
 {'id': 1665,
  'gender': 'Female',
  'age': 79.0,
  'hypertension': 1,
  'heart_disease': 0,
  'ever_married': 'Yes',
  'work_type': 'Self-employed',
  'Residence_type': 'Rural',
  'avg_glucose_level': 174.12,
  'bmi': 24.0,
  'smoking_status': 'never smoked',
  'stroke': 1},
 {'id': 10434,
  'gender': 'Female',
  'age': 69.0,
  'hypertension': 0,
  'heart_disease': 0,
  'ever_married': 'No',
  'work_type': 'Private',
  'Residence_type'

Ajouter des valeurs par défault et les conditions pour chaque filtre.

Pour les types, on indique qu'il s'agit de paramètres optionels en utilisant le module python _typing_

```
from typing import Optional
def filter_patient(stroke_data_df: pd.DataFrame, gender: Optional[str] = None,etc)
```

Adapter les types en utilisant ce modèle.

In [42]:
# fonction avec ajout de paramètres par défault et de type
def filter_patient(df, gender=None, stroke=None, max_age=None)->list[dict]:
    
    if stroke is not None :
        df = df.loc[df['stroke'] == stroke ]
    if gender is not None :
        df = df.loc[df['gender'] == gender]
    if max_age is not None : 
        df = df.loc[df['age'] <= max_age]
    return df.to_dict(orient="records")

filter_patient(df_imputed, 'Female')

[{'id': 51676,
  'gender': 'Female',
  'age': 61.0,
  'hypertension': 0,
  'heart_disease': 0,
  'ever_married': 'Yes',
  'work_type': 'Self-employed',
  'Residence_type': 'Rural',
  'avg_glucose_level': 202.21,
  'bmi': 30.97008547008547,
  'smoking_status': 'never smoked',
  'stroke': 1},
 {'id': 60182,
  'gender': 'Female',
  'age': 49.0,
  'hypertension': 0,
  'heart_disease': 0,
  'ever_married': 'Yes',
  'work_type': 'Private',
  'Residence_type': 'Urban',
  'avg_glucose_level': 171.23,
  'bmi': 34.4,
  'smoking_status': 'smokes',
  'stroke': 1},
 {'id': 1665,
  'gender': 'Female',
  'age': 79.0,
  'hypertension': 1,
  'heart_disease': 0,
  'ever_married': 'Yes',
  'work_type': 'Self-employed',
  'Residence_type': 'Rural',
  'avg_glucose_level': 174.12,
  'bmi': 24.0,
  'smoking_status': 'never smoked',
  'stroke': 1},
 {'id': 10434,
  'gender': 'Female',
  'age': 69.0,
  'hypertension': 0,
  'heart_disease': 0,
  'ever_married': 'No',
  'work_type': 'Private',
  'Residence_type'

Tester la fonction sans argument pour les filtres, elle doit donc renvoyer le dataframe non filtré.

In [43]:
# test fonction sans argument pour les filtres
def filter_patient(df, gender=None, stroke=None, max_age=None)->list[dict]:
    if stroke is not None :
        df = df.loc[df['stroke'] == stroke ]
    if gender is not None :
        df = df.loc[df['gender'] == gender]
    if max_age is not None : 
        df = df.loc[df['age'] <= max_age]
    return df.to_dict(orient='records')

filter_patient(df_imputed)

[{'id': 9046,
  'gender': 'Male',
  'age': 67.0,
  'hypertension': 0,
  'heart_disease': 1,
  'ever_married': 'Yes',
  'work_type': 'Private',
  'Residence_type': 'Urban',
  'avg_glucose_level': 228.69,
  'bmi': 36.6,
  'smoking_status': 'formerly smoked',
  'stroke': 1},
 {'id': 51676,
  'gender': 'Female',
  'age': 61.0,
  'hypertension': 0,
  'heart_disease': 0,
  'ever_married': 'Yes',
  'work_type': 'Self-employed',
  'Residence_type': 'Rural',
  'avg_glucose_level': 202.21,
  'bmi': 30.97008547008547,
  'smoking_status': 'never smoked',
  'stroke': 1},
 {'id': 31112,
  'gender': 'Male',
  'age': 80.0,
  'hypertension': 0,
  'heart_disease': 1,
  'ever_married': 'Yes',
  'work_type': 'Private',
  'Residence_type': 'Rural',
  'avg_glucose_level': 105.92,
  'bmi': 32.5,
  'smoking_status': 'never smoked',
  'stroke': 1},
 {'id': 60182,
  'gender': 'Female',
  'age': 49.0,
  'hypertension': 0,
  'heart_disease': 0,
  'ever_married': 'Yes',
  'work_type': 'Private',
  'Residence_type'

In [44]:
# testing filter function
def get_info_by_id(df, patient_id: int):
        
    if patient_id is not None:
        df = df.loc[df['id'] == patient_id]
        
    return df.to_dict(orient='records')
get_info_by_id(df_health, 9046)

[{'id': 9046,
  'gender': 'Male',
  'age': 67.0,
  'hypertension': 0,
  'heart_disease': 1,
  'ever_married': 'Yes',
  'work_type': 'Private',
  'Residence_type': 'Urban',
  'avg_glucose_level': 228.69,
  'bmi': 36.6,
  'smoking_status': 'formerly smoked',
  'stroke': 1}]

Cette fonction va être utilisée dans la définition de l'API pour créer une route qui permette d'accéder à des données filtrées sur les patients.

Dans le fichier de définition de l'API, toutes les fonctions vont travailler sur les données du fichier. 

Pour alléger les fonctions on va donc utiliser une **variable globale** pour les données et supprimer le paramètre `df` de la fonction.

On lit les données en début de fichier puis on travaille au sein des fonctions sur une copie du dataframe de données.


**En résumé les modifications à faire sont :**


- Supprimer le paramètre df de la fonction,
- Ajouter en début de fonction :  
```df = stroke_data_df.copy()```

1. Dans le fichier filters.py, il suffit d'ajouter : 
- lecture du fichier de données prétraitée dans la variable *df* en début de fichier (utiliser pandas),
- @app.get("/patients/") pour définir le route,
puis la fonction.

2. Dans le fichier api.py: appeler la fonction dans la route correspondante.

Tester la route avec 

```poetry run fastapi dev stroke_api/main.py```

http://127.0.0.1:8000/docs : utiliser la fonctionnalité Try it out pour tester la route.

---
## Autres routes

De la même manière, créer les fonctions appropriées pour la création de :
- la route `/patients/{id}` : Récupère les détails d’un patient donné (via son identifiant unique) 

- la route `/stats/` : Fournit des statistiques agrégées sur les patients (ex. : nb total de patients, âge moyen, taux d’AVC, répartition hommes/femmes).

- Lister les tâches à faire sous forme d'issue github : travailler sur une branche différentes pour l'ajout de chacune des routes.